# Exploratory Data Analysis

This notebook introduces you to exploratory data analysis using the [pandas](https://pandas.pydata.org) library. Pandas is not specific to geospatial data, so what we do here can be applied to pretty much any dataset you encounter.

![pandas logo](https://pandas.pydata.org/pandas-docs/stable/_static/pandas.svg)

As a quick intro, we reommend [10 minutes to pandas](https://pandas.pydata.org/docs/user_guide/10min.html). This notebook will point you to the relevant sections of the [pandas user guide](https://pandas.pydata.org/docs/user_guide/index.html), which is generally a great resource whenever you need documentation about a pandas feature.

Let's get started by loading pandas:

In [ ]:
import pandas as pd

### Loading data

Load some traffic accident data directly from the web ([NYC Crash Mapper](http://crashmapper.org/#/?cfat=true&cinj=true&endDate=2019-03&geo=citywide&identifier=&lngLats=%255B%255D&mfat=true&minj=true&noInjFat=false&pfat=true&pinj=true&startDate=2018-03)):

In [ ]:
url="https://chekpeds.carto.com/api/v2/sql?q=SELECT%20c.cartodb_id%2C%20c.socrata_id%2C%20c.the_geom%2C%20c.on_street_name%2C%20c.cross_street_name%2C%20c.date_val%20AS%20date_time%2C%20c.latitude%2C%20c.longitude%2C%20c.borough%2C%20c.zip_code%2C%20c.crash_count%2C%20c.number_of_cyclist_injured%2C%20c.number_of_cyclist_killed%2C%20c.number_of_motorist_injured%2C%20c.number_of_motorist_killed%2C%20c.number_of_pedestrian_injured%2C%20c.number_of_pedestrian_killed%2C%20c.number_of_persons_injured%2C%20c.number_of_persons_killed%2C%20array_to_string(c.contributing_factor%2C%20%27%2C%27)%20as%20contributing_factors%2C%20array_to_string(c.vehicle_type%2C%20%27%2C%27)%20as%20vehicle_types%20FROM%20crashes_all_prod%20c%20%20WHERE%20(%20year%3A%3Atext%20%7C%7C%20LPAD(month%3A%3Atext%2C%202%2C%20%270%27)%20%3C%3D%20%272019%27%20%7C%7C%20LPAD(3%3A%3Atext%2C%202%2C%20%270%27)%20)%20AND%20(%20year%3A%3Atext%20%7C%7C%20LPAD(month%3A%3Atext%2C%202%2C%20%270%27)%20%3E%3D%20%272018%27%20%7C%7C%20LPAD(3%3A%3Atext%2C%202%2C%20%270%27)%20)%20%20AND%20(%20number_of_cyclist_injured%20%3E%200%20OR%20number_of_motorist_injured%20%3E%200%20OR%20number_of_pedestrian_injured%20%3E%200%20%20OR%20%20number_of_cyclist_killed%20%3E%200%20OR%20number_of_motorist_killed%20%3E%200%20OR%20number_of_pedestrian_killed%20%3E%200%20)&format=CSV"
accidents = pd.read_csv(url)

This creates a [dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), let's take a glimpse:

In [ ]:
accidents

Loading a local file is just as easy:

In [ ]:
accidents = pd.read_csv("accidents.csv") # <- this is actually an older version of the same dataset we loaded from the web before
accidents

Let's check out the [data types](https://pbpython.com/pandas_dtypes.html) on our accidents data

In [ ]:
accidents.dtypes

Strings and the timestamp are stored as [objects](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html?highlight=object) – we'll fix that in a bit.

Let's try to access parts of the accidents DataFrame. First, a single column. This will return a pandas [Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html).

In [ ]:
accidents['vehicle_types']

Slicing rows – returns a new DataFrame:

In [ ]:
accidents[2:10] # rows 2 to 9 (!)

In [ ]:
accidents[2:]   # all rows from row 2

In [ ]:
accidents[:10]  # all rows up to row 9

### Using [.loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html)

Access a group of rows and columns by label(s) or a boolean array. ``.loc[]`` is primarily label based, but may also be used with a boolean array.

It returns the 3rd row of the dataframe!

In [ ]:
accidents.loc[3]

Using .loc to pick columns:

In [ ]:
accidents.loc[27:29, ['number_of_cyclist_injured', 'number_of_cyclist_killed']]

### Using [.iloc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html)

... to get the same result. Notice how .loc and .iloc use different indexing – .loc is *inclusive* the last mentioned index (29 here), .iloc is *exlusive* (30 in this example). Here's a [good article](https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/) comparing the different access methods.

In [ ]:
accidents.iloc[27:30, 11:13]

### DataFrame indexes

Each dataframe has an [index](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing). Initially, that's just a number for each row:

Indexing serves many purposes:

- Identifies data (i.e. provides metadata) using known indicators, important for analysis, visualization, and interactive console display.
- Enables automatic and explicit data alignment.
- Allows intuitive getting and setting of subsets of the data set.

In [ ]:
accidents.index

Let's get an overview of the columns in the dataset (again)...

In [ ]:
accidents.columns

... and index the dataset by timestamp of the accident instead:

In [ ]:
accidents.set_index(pd.to_datetime(accidents['date_time']), inplace=True)
accidents.set_index(accidents.index.tz_localize(None), inplace=True)
accidents.index

Now we can easily query the accidents by date range:

In [ ]:
accidents.sort_index().loc['2019-01-01':'2019-02-01']

How many accidents do we have per month of the year?

In [ ]:
accidents.resample('ME').sum()

..and per hour?

In [ ]:
accidents.resample('h').sum()  

and per Day?

In [ ]:
accidents.resample('D').sum()

How many accidents are there on average per hour of the day?

In [ ]:
hourly_accidents = accidents.resample('h').sum()
hourly_accidents['hour'] = hourly_accidents.index.hour
hourly_accidents.groupby(['hour']).mean(numeric_only=True) # filtering out some non-numeric entries

Let's plot this:

In [ ]:
# you only need this one once – all plots from here on will be embedded in the notebook
%matplotlib inline 

mean_hourly_accidents = hourly_accidents.groupby(['hour']).mean(numeric_only=True)
mean_hourly_accidents.plot(y='crash_count')

# 🏋 Exercise

**Which month of the year has the [highest number](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.max.html) of accidents?**

In [ ]:
# your code here...



### Boolean indexing

With [boolean indexing](https://www.geeksforgeeks.org/boolean-indexing-in-pandas/), we can select rows where certain criterea (or combinations thereof) are true. As an example, let's select all accidents where cyclists have been injured:

In [ ]:
# it returns for each row whether the criterion is valid 
accidents['number_of_cyclist_injured'] > 0

Filter the dataframe using this boolean series:

In [ ]:
# it returns the outcome after applying the criterion

accidents[accidents['number_of_cyclist_injured'] > 0]

Combine multiple citeria:

In [ ]:
accidents[(accidents['number_of_cyclist_injured'] > 0) | (accidents['number_of_cyclist_killed'] > 0)]

Note: The operators are: ``|`` for OR, ``&`` for AND, and ``~`` for NOT. 

In [ ]:
accidents[ ~ (accidents['number_of_cyclist_injured'] > 0)]

Let's query using string matching:

In [ ]:
accidents[accidents['vehicle_types'].str.contains('Taxi')]

Ooops... Do we have null values in our ```vehicle_types``` column?

In [ ]:
accidents[accidents['vehicle_types'].isnull()]

Let's replace the NaN values with an empty string:

In [ ]:
accidents.vehicle_types.fillna('', inplace=True)
accidents[accidents['vehicle_types'].isnull()]

Now this should work:

In [ ]:
accidents[accidents['vehicle_types'].str.contains('Taxi')]

Writing the filtered dataset to a CSV file:

In [ ]:
taxi_accidents = accidents[accidents['vehicle_types'].str.contains('Taxi')]
taxi_accidents.to_csv('taxi_accidents.csv')

### Plotting and (simple) mapping

In [ ]:
accidents.plot(x='longitude', 
               y='latitude', 
               kind='scatter')

These accidents are all in NYC, so the dots at the top left and bottom right are clearly wrong coordinates. Let's remove them:

In [ ]:
accidents = accidents[ (accidents['latitude'] > 30) & (accidents['longitude'] > -100)  ]
accidents.plot.scatter(x='longitude', 
               y='latitude')

Not very pretty, but it's a start. 

Let's make a fancy map now. See the docs for explanation of the [variables](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html#pandas.DataFrame.plot) and [color scales](https://matplotlib.org/users/colormaps.html).

In [ ]:
accidents[accidents['number_of_persons_injured'] > 0].plot.scatter(x='longitude', 
               y='latitude', 
               c='number_of_persons_injured',
               s=0.7,
               title='NYC accidents with injuries',                                                   
               colormap='Wistia',
               figsize=[14,14])

In [ ]:
help(accidents.plot.scatter)

Let's take a look at hexbins to identify potential hotspots:

In [ ]:
accidents.plot.hexbin(x='longitude', 
               y='latitude',
               figsize=[14,14])

Let's make a KDE map using [Seaborn](https://seaborn.pydata.org). If you don't know what KDE is, here is an excellent interactive tutortial explaning the idea of [Kernel Density Estimation](https://mathisonian.github.io/kde/).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# in case we have duplicate columns, which was the case, as we duplicated the 'Date_time' column
#accidents = accidents.reset_index(drop=True)

In [ ]:
plt.figure(figsize=(14, 14))


sns.kdeplot(x=accidents['longitude'], 
            y=accidents['latitude'],
            cmap='Reds',
            fill=True)
plt.savefig('kde.pdf') # saves the plot to a PDF file in the same folder as this notebook 
                       # – this command has to go in the same cell that produces the plot!

## Exercise

Try to make the map more detailed – see the [Seaborn docs](https://seaborn.pydata.org/generated/seaborn.kdeplot.html)

Let's get an overview of the accidents dataset:

In [ ]:
accidents.describe()

How does the distribution of monthly fatalities look like?

In [ ]:
monthly_accidents = accidents.resample('ME').sum()

plt.figure(figsize=(14, 5))
sns.boxplot(monthly_accidents['number_of_persons_killed'])



🤔 How do you [read a box plot again](https://en.wikipedia.org/wiki/Box_plot#Visualization)? 

Let's make a distribution plot:

In [ ]:
plt.figure(figsize=(14, 8))
sns.histplot(monthly_accidents['number_of_persons_killed'], kde=True)



Scatter plot with fitted linear regression line:

In [ ]:
sns.lmplot(x='number_of_persons_injured',y='number_of_persons_killed',data=monthly_accidents)

In [ ]:
monthly_accidents = accidents.resample('ME').sum()
monthly_accidents['month'] = monthly_accidents.index.month
print(monthly_accidents.max())

%matplotlib inline 

monthly_accidents.plot(y='crash_count')